<a href="https://colab.research.google.com/github/dipsankar-sinha/Wav2Vec2-ASR-Amharic-CV-16.1/blob/main/Fine_Tuning_Wav2Vec2_CV_Amharic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine Tuning Wav2Vec2 for bulding an ASR model using Mozila Common Voice Amharic Dataset:16.1**
---
Date:24/06/2024
Created by: Dipsankar Sinha, MCA, KIIT-DU
---
**Reference: https://huggingface.co/blog/fine-tune-xlsr-wav2vec2**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%%capture
!pip install huggingface_hub
!pip install librosa
!pip install jiwer
!pip install --upgrade pyarrow
!pip install --upgrade huggingface_hub
!pip install transformers==4.42.0
!pip install torch==2.3.0+cu121 -f https://download.pytorch.org/whl/cu121/torch_stable.html
!pip install datasets==2.20.0
!pip install tokenizers==0.19.1

In [3]:
!pip install torchvision==0.18
!pip install torchaudio==2.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.0+cu121
    Uninstalling torchvision-0.19.0+cu121:
      Successfully uninstalled torchvision-0.19.0+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.0+cu121
    Uninstalling torchaudio-2.4.0+cu121:
      Successfully uninstalled torchaudio-2.4.0+cu121


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 15 16:57:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
from huggingface_hub import notebook_login

notebook_login()


In [52]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [53]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_16_1","am",split="train+validation")
common_voice_test = load_dataset("mozilla-foundation/common_voice_16_1","am",split="test")

In [54]:
common_voice_train

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 638
})

In [55]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes","variant"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes","variant"])

In [56]:
common_voice_test

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 162
})

In [57]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


In [58]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]), num_examples=10)

,sentence
0,በቀዳሚዎቹ ዘመናት አባታችን አዳም አትብላ የተባለውን ዕጸ በለስ በልቶ የፈጣሪን ትእዛዝ ተላለፈ
1,ነገር ግን ሚኒስትሩ ለቢቢሲ እንደገለጹት ጄነራሎቹ የአገሪቷን የመሪነት ስፍራ ይቆናጠጣሉ የሚል ግምት እንዳልነበራቸው ነው።
2,ትኩረትን መስጠት መንፈሳዊነትን ማጎልበት በጥሞና ዘለግ ላለ ሰዓት መቆየት እና ራስን ማሻሻል ነው ይላሉ በመጽሐፋቸው።
3,በወንዶች የማራቶን ውድድር ቀነኒሳ በቀለን ጨምሮ በርቀቱ የዓለማችን ፈጣን የሆኑ አራት አትሎቶች ተሳታፊ ይሆናሉ።
4,የብረት ባለሙያ የነበሩት ሉላ ሲልቫ አምስት መቶ ሰማኒያ ቀናትን በእስር አሰልፈዋል።
5,የቤተሰብ አባላት ቀብራቸውን ለመፈፀም የሚያስችል ፈቃድ ከሚመለከታቸው የመንግሥት ኃላፊዎች ማግኘት አልቻሉም።
6,በረመዳን ወቅት ልምምድ ስታደርግ የነበረችው ሲፈን በመጀመሪያ የማራቶን ውድድሯ ታሸንፋለች ተብሎ አልተጠብቅም ነበር።
7,ለምን የተለዩ እንደሆኑ ገና አልታወቀም ይላሉ ፕሮፌሰሯ።
8,በተለይ በጠዋቱ ክፍለ ጊዜ የሰውነት እንቅስቃሴ ማድረግ ከባድ ሊመስል ይችላል።
9,ነገር ግን መጠኑን መወሰን ተገቢ ነው ሲሉ ይመክራሉ።


In [59]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"፡።፣፤፦፧፨፠]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]) + " "
    return batch

In [60]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [61]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,በተለይ ሲጋራ ስጡኝ ማለት ሲያቆሙ ቤተሰቡ ግራ ተጋብቷል
1,ይኽ የሚሆነው ግን እንደ ኒቆዲሞስ ተስፋን ሰንቀው ወደ ትንሣኤው ብርሃን የሚገሠግሡ ናቸው
2,ፓስተር ማኬንዚ ከአንድ ሳምንት በፊት በቁጥጥር ሥር ውሏል
3,በኋላ ታዲያ አገረ ገዢው ኪሱ ሲገባ ቦርሳው እንደጠፋበት አወቀ
4,አንድ ምሥል ደግሞ ሕንጻ ጣሪያ ላይ የወጣ መኪና ያሳያል
5,የማኅበረሰብ አንቂዎችን ዒላማ ያደረገ እስር እና ወከባ በእጅጉ አሳሳስቦኛል ብሎ ነበር
6,የማሪዩፖል ከተማ ለወራት ከዘለቀው መራራ ጦርነት በኋላ በሩሲያ ወታደሮች ቁጥጥር ስር ውስጥ ገብታለች
7,ቾይ የፕሬዝደንት ፓርክን መልካም ግንኙነት ተጠቅመው ከባለሃብቶች ጋር ሚሊዮን ዶላሮችን በእርዳታ መልክ ማግኘት ችለዋል
8,እርሷም አዎ አለችው
9,እያዩ መሰወር እዛው ሆኖ ማጣት


In [62]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [63]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)


Map:   0%|          | 0/638 [00:00<?, ? examples/s]

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

In [64]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ፕ': 0,
 'ዑ': 1,
 'እ': 2,
 'ዳ': 3,
 'ሠ': 4,
 'ቀ': 5,
 'ቧ': 6,
 'ፉ': 7,
 'ዤ': 8,
 'ጲ': 9,
 'ው': 10,
 'ዙ': 11,
 'ሬ': 12,
 'ሳ': 13,
 'ቼ': 14,
 'ጄ': 15,
 'ሚ': 16,
 'ቦ': 17,
 'ጁ': 18,
 'ዛ': 19,
 'ባ': 20,
 'ጶ': 21,
 'ዣ': 22,
 'ጧ': 23,
 'ጪ': 24,
 'ን': 25,
 'ኔ': 26,
 'ቫ': 27,
 'ሸ': 28,
 'ጠ': 29,
 'ኢ': 30,
 'ኪ': 31,
 'ላ': 32,
 'ብ': 33,
 'ቾ': 34,
 'ገ': 35,
 'ሽ': 36,
 'ሾ': 37,
 'ቅ': 38,
 'ጮ': 39,
 'ቸ': 40,
 'ጸ': 41,
 'ቹ': 42,
 'ኖ': 43,
 'ሄ': 44,
 'ሯ': 45,
 'ና': 46,
 'ኤ': 47,
 'ዟ': 48,
 'ኑ': 49,
 'ጣ': 50,
 'ዮ': 51,
 'ቶ': 52,
 'ዐ': 53,
 'ሙ': 54,
 'ዞ': 55,
 'ጤ': 56,
 'ማ': 57,
 '“': 58,
 'ኃ': 59,
 'ሔ': 60,
 'አ': 61,
 'ታ': 62,
 'ሑ': 63,
 'ተ': 64,
 'ጊ': 65,
 'ጹ': 66,
 'ሎ': 67,
 'ኛ': 68,
 'ኩ': 69,
 'ቃ': 70,
 'ቡ': 71,
 'ት': 72,
 'ዒ': 73,
 'ጌ': 74,
 'ኬ': 75,
 'ሮ': 76,
 'ቷ': 77,
 'ሼ': 78,
 'ጿ': 79,
 'ስ': 80,
 'ጉ': 81,
 'ጾ': 82,
 'ኳ': 83,
 'ጂ': 84,
 'ጽ': 85,
 'ህ': 86,
 'ዚ': 87,
 'በ': 88,
 'ር': 89,
 'ሞ': 90,
 'ፖ': 91,
 'ች': 92,
 'ዜ': 93,
 'ቆ': 94,
 'ፋ': 95,
 'ቴ': 96,
 'ፍ': 97,
 'ፈ': 98,
 'ፑ': 99,
 'ኙ': 100,

In [65]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [66]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

230

In [67]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [68]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [69]:
repo_name = "wav2vec2-large-xls-r-300m-amharic-demo-colab"

In [70]:
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/2.55k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/DipsankarSinha/wav2vec2-large-xls-r-300m-amharic-demo-colab/commit/530bd5dfdc179ce206121fc460a1427b2c0c3268', commit_message='Upload tokenizer', commit_description='', oid='530bd5dfdc179ce206121fc460a1427b2c0c3268', pr_url=None, pr_revision=None, pr_num=None)

In [71]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [72]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [73]:
common_voice_train[0]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/9fd4b5e65c558d66e65957babf9ed97d68641d9ebd55b629a42ac01f925914d7/am_train_0/common_voice_am_37761292.mp3'

In [74]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/9fd4b5e65c558d66e65957babf9ed97d68641d9ebd55b629a42ac01f925914d7/am_train_0/common_voice_am_37761292.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.17188138e-05,  3.26943336e-06, -1.98085672e-06]),
 'sampling_rate': 48000}

In [75]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [76]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/9fd4b5e65c558d66e65957babf9ed97d68641d9ebd55b629a42ac01f925914d7/am_train_0/common_voice_am_37761292.mp3',
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.02513113e-05,  3.72633804e-06,  6.73267641e-06]),
 'sampling_rate': 16000}

In [77]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


በኢትዮጵያ ማኅበራዊ ሚዲያዎች ላይ የተጣለው ገደብ እንዲነሳ አምነስቲ ጠየቀ 


In [78]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])


Target text: ለምን የተለዩ እንደሆኑ ገና አልታወቀም ይላሉ ፕሮፌሰሯ 
Input array shape: (86976,)
Sampling rate: 16000


In [79]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


In [80]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)


In [81]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [82]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [83]:
%%capture
!pip install evaluate

In [84]:
import evaluate

# Load the WER metric
wer_metric = evaluate.load("wer")

In [85]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [86]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2284: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [88]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=60,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=100,
  save_total_limit=2,
  push_to_hub=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [89]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2284: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [90]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [91]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,12.694800,4.162145,1.000000
200,4.102600,4.036461,1.000000
300,4.003700,3.972565,1.000667
400,3.948500,3.952430,1.000667
500,3.463500,2.438361,0.997999
600,1.170900,1.698669,0.945297
700,0.495500,1.592720,0.907272
800,0.316300,1.674962,0.883256
900,0.237200,1.668306,0.881254
1000,0.189600,1.655454,0.877919


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version

TrainOutput(global_step=1200, training_loss=2.5776540406545005, metrics={'train_runtime': 5921.022, 'train_samples_per_second': 6.465, 'train_steps_per_second': 0.203, 'total_flos': 7.42468605180586e+18, 'train_loss': 2.5776540406545005, 'epoch': 60.0})

In [92]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/DipsankarSinha/wav2vec2-large-xls-r-300m-amharic-demo-colab/commit/69c7a7d44f82f9b31d757e272fc6ce7acf32be2c', commit_message='End of training', commit_description='', oid='69c7a7d44f82f9b31d757e272fc6ce7acf32be2c', pr_url=None, pr_revision=None, pr_num=None)

In [93]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("DipsankarSinha/wav2vec2-large-xls-r-300m-amharic-demo-colab").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("DipsankarSinha/wav2vec2-large-xls-r-300m-amharic-demo-colab")


config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.12k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [94]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [95]:
common_voice_test_transcription = load_dataset("mozilla-foundation/common_voice_16_1","am", split="test")

In [96]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["sentence"].lower())


Prediction:
ተከታዮቹም በተነገራቸው መሠራት የተባሉትን ካደረጉ በኋላ በራኅ ብይወታቸዋልፏል

Reference:
ተከታዮቹም በተነገራቸው መሠረት የተባሉትን ካደረጉ በኋላ በረሃብ ሕይወታቸው አልፏል።


In [97]:
# from transformers import AutoModelForCTC, Wav2Vec2Processor

# model = AutoModelForCTC.from_pretrained("DipsankarSinha/wav2vec2-large-xls-r-300m-amharic-demo-colab").to("cuda")
# processor = Wav2Vec2Processor.from_pretrained("DipsankarSinha/wav2vec2-large-xls-r-300m-amharic-demo-colab")

In [98]:
import torch
import pickle
torch.save(model.state_dict(), "amharic_fine_tuned_wav2vec2.pth")
with open("amharic_fine_tuned_wav2vec2_processor.pkl", "wb") as f:
  pickle.dump(processor, f)

In [99]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

results = common_voice_test.map(map_to_result, remove_columns=common_voice_test.column_names)

Parameter 'function'=<function map_to_result at 0x7801754caef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/162 [00:00<?, ? examples/s]

In [100]:
print("Test WER:",wer_metric.compute(predictions=results["pred_str"], references=results["text"]))

Test WER: 0.7805203468979319


In [101]:
show_random_elements(results)

,pred_str,text
0,ልክየሱስ ክርስቶስ እንዳደረገው የመስከሉ እናግዳሚ እንጨተሸክም ወደ መስካያው ስፍራ እንዲወስጥ ይገደዳል,ልክ ኢየሱስ ክርስቶስ እንዳደረገው የመስቀሉን አግዳሚ እንጨት ተሸክሞ ወደ መሰቀያው ስፍራ እንዲወስድ ይገደዳል
1,ጀነራሎች ሱዳንን የማስተዳድርስልጣለ ሲቪል አስትዳደር ለማስጠላላፍ ጠስማምተናብለዋል,ጀነራሎቹ ሱዳንን የማስተዳደር ስልጣን ለሲቪል አስተዳደር ለማስተላለፍ ተስማምተናል ብለዋል
2,በባህሉ መሰረት ስጋ የሚበለው መጨረሻ ላይ,በባህሉ መሰረት ስጋ የሚበላው መጨረሻ ላይ ነው
3,ድምጽዊው ለረጅም ግዜ በአደርበ ትመም በህክም ና ሴረዳቆይቷል,ድምፃዊው ለረዥም ጊዜ ባደረበት ህመም በህክምና ሲረዳ ቆይቷል
4,ዳያ አሁን የየኒቨርስቲት ምህርቷ ለመጀመር ዋዜ ማወረይበት ሆነም አሁንም በተላዩ ተግባራት ተጠምዳለች,ዳያ አሁን የዩኒቨርስቲ ትምህርቷን ለመጀመር ዋዜማው ላይ ብትሆንም አሁንም በተለያዩ ተግባራት ተጠምዳለች
5,የታለጵ የኢትዮጵያዳ ሲ ግድብየኢትዮጵያ ግስተም እራብ ከሱዳን እጋር በሉ ፕቦ አሰንበት ጎቢዮን ሻንጉል ጉሙስክልል በጉባ ወረ ዳ ይቀኛል,የታላቁ የኢትዮጵያ ሕዳሴ ግድብ ኢትዮጵያ በስተምዕራብ ከሱዳን ጋር በምትዋሰንበት የቤኒሻንጉል ጉሙዝ ክልል በጉባ ወረዳ ይገኛል
6,እነርሱም አዎ አሉት,እነርሱም አዎ አሉት
7,ዜጎች እለማስወጣህም ወሰኑን አንድ ሰማቸው ያተገልቀ ያሜ ሪካመራ ለአሶሽ ይትርፕረሥ ተነግረዋል,ዜጎችን ለማስወጣት መወሰኑን አንድ ስማቸው ያልተገለጸ የአሜሪካ አመራር ለአሶሽየትድ ፕረስ ተናግረዋል
8,እንግስትተወጡ ድርጅት ሰብዋዊ መውቶችን ምክር በይታባልጥሆነምየሰብኋዊ መብቶችንና የእምንህ ንጻረተን በመጥስት ከስሳለች,የመንግሥታቱ ድርጅት የሰብአዊ መብቶች ምክር ቤት አባል ብትሆንም የሰብአዊ መብቶችን እና የእምነት ነፃነትን በመጣስ ትከሰሳለች
9,የሲኒያውሽ እና አተልብሻችው ዩበር አብበባምን ካቱ ከፓላማ ንጅወጦ ተነረጉ,የኬንያዋ ሴናተር ልብሳቸው የወር አበባ በመንካቱ ከፓርላማ እንዲወጡ ተደረጉ


# **Creating an Web App Using Gradio to check and test the model**
---
Date:31/07/2024
Created by: Dipsankar Sinha, MCA, KIIT-DU
---
Reference: https://www.youtube.com/watch?v=jG52ot4njNs

In [102]:
%%capture
!pip install gradio -q

In [103]:
import gradio as gr
import torch
from transformers import AutoModelForCTC, Wav2Vec2Processor
import librosa
import pickle

In [104]:
# Load your saved model and processor
model = AutoModelForCTC.from_pretrained("DipsankarSinha/wav2vec2-large-xls-r-300m-amharic-demo-colab")
model.load_state_dict(torch.load("amharic_fine_tuned_wav2vec2.pth"))
model.eval()  # Set the model to evaluation mode
model.to("cuda")  # Move the model to GPU if available
with open("amharic_fine_tuned_wav2vec2_processor.pkl", "rb") as f:
    processor = pickle.load(f)

In [105]:
# Define the ASR function
def transcribe(audio):
    # Load audio file
    audio_input, _ = librosa.load(audio, sr=16000)

    # Preprocess audio
    input_values = processor(audio_input, return_tensors="pt", padding="longest").input_values.to("cuda")

    # Get model predictions
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode predictions
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

In [106]:
# Create the Gradio interface
inputs = gr.Audio(type="filepath")
outputs = gr.Textbox(label="Transcription")

gr_interface = gr.Interface(
    fn=transcribe,
    inputs=inputs,
    outputs=outputs,
    title="Amharic Automatic Speech Recognition",
    description="Transcribe Amharic speech using a fine-tuned Wav2Vec2 model."
)

gr_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5143f20c7c09983b73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
